# Afinamiento de modelos de lenguaje para dominios específicos

## Paso 1: Fine tuning de un modelo con dataset tipo 'instruct'

El ajuste fino con datos tipo "instruct" adapta los modelos de lenguaje (LLMs) para responder eficazmente a instrucciones específicas.

### **Características clave**:
1. **Entrenamiento con pares de instrucciones y respuestas**: Se utilizan ejemplos como "Resume este texto en dos frases" y su respuesta esperada, para enseñar al modelo a seguir comandos.
2. **Datos etiquetados o supervisados**: Las respuestas pueden ser creadas por humanos o generadas por modelos existentes y revisadas manualmente.

### **Proceso**:
1. **Creación del dataset**: Se compilan instrucciones y respuestas alineadas con los objetivos del usuario.
2. **Ajuste fino supervisado (SFT)**: El modelo se entrena utilizando estos pares para mejorar su capacidad de responder a comandos de manera precisa y alineada con la intención del usuario.

In [1]:
!pip install bitsandbytes==0.44.1 > /dev/null 2>&1
!pip install datasets==3.1.0 > /dev/null 2>&1
!pip install peft==0.13.2 > /dev/null 2>&1
!pip install wandb==0.18.7 > /dev/null 2>&1
!pip install trl==0.12.1 > /dev/null 2>&1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer, SFTConfig
import wandb
import os
import warnings

os.environ['WANDB_NOTEBOOK_NAME'] = 'nb01-instruct.ipynb'

### 1. Importar el modelo ya pre-entrenado y el tokenizador

El modelo `unsloth/mistral-7b-instruct-v0.2-bnb-4bit` es una variante compacta del modelo Mistral de 7 mil millones de parámetros, diseñada para tareas de instrucción. Utiliza técnicas de cuantización en 4 bits (bnb-4bit) para optimizar el uso de memoria y permitir su ejecución en hardware menos potente sin perder demasiada precisión. Este modelo está ajustado para interpretar y responder instrucciones en lenguaje natural, siendo útil para aplicaciones en procesamiento de lenguaje natural (NLP) que requieren respuestas eficientes y precisas en un entorno optimizado.

In [ ]:
warnings.filterwarnings("ignore")

model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
tokenizer_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map='auto')

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

### 2. Carga de los datos de entrenamiento

In [ ]:
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/training/data/list_of_strings.jsonl', split="train")


def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=64)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])


wandb.init(
    project='LLM_training',
    name=model_name + '_autoregressive_instruct_fine_tuning'
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find nb01-instruct.ipynb.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### 3. Configuración del entrenamiento mediante LoRA y SFT

- **LoRA (Low-Rank Adaptation):**  
LoRA es una técnica utilizada para ajustar modelos grandes de lenguaje de manera eficiente. En lugar de actualizar todos los parámetros del modelo durante el entrenamiento, LoRA introduce matrices adicionales de bajo rango que capturan los cambios necesarios. Esto reduce significativamente el costo computacional y de memoria, permitiendo un ajuste fino (fine-tuning) eficiente sin necesidad de almacenar o modificar todos los parámetros originales del modelo.

- **SFT (Supervised Fine-Tuning):**  
SFT se refiere al ajuste fino de un modelo de lenguaje utilizando datos etiquetados de manera supervisada. Este proceso entrena el modelo para realizar tareas específicas mediante ejemplos claros de entrada y salida, como preguntas y respuestas, traducción, o clasificación. Es un paso clave para especializar modelos generales en tareas concretas o dominios específicos.

In [ ]:
# Configuración de LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # rango de LoRA
    lora_alpha=16,  # hiperparámetro de LoRA
    lora_dropout=0.1,  # dropout de LoRA
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']  # módulos objetivo para aplicar LoRA
)

model = get_peft_model(model, lora_config)

# Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=1e-4,
    fp16=True,
    evaluation_strategy="no",
    eval_steps=10_000,
    report_to="wandb"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    data_collator=data_collator,
)

### 4. Entrenamiento del modelo

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
100,1.689700
200,0.621500
300,0.570400
400,0.508600
500,0.450200
600,0.358600
700,0.256200


TrainOutput(global_step=750, training_loss=0.6088946825663248, metrics={'train_runtime': 808.3992, 'train_samples_per_second': 1.856, 'train_steps_per_second': 0.928, 'total_flos': 4107819810816000.0, 'train_loss': 0.6088946825663248, 'epoch': 10.0})

### 5. Guardar el modelo

In [ ]:
wandb.finish()

trainer.save_model("./drive/MyDrive/training/models/sft_lora_multiples_of_three")
tokenizer.save_pretrained("./drive/MyDrive/training/models/sft_lora_multiples_of_three")

('./drive/MyDrive/training/models/sft_lora_multiples_of_three/tokenizer_config.json',
 './drive/MyDrive/training/models/sft_lora_multiples_of_three/special_tokens_map.json',
 './drive/MyDrive/training/models/sft_lora_multiples_of_three/tokenizer.model',
 './drive/MyDrive/training/models/sft_lora_multiples_of_three/added_tokens.json',
 './drive/MyDrive/training/models/sft_lora_multiples_of_three/tokenizer.json')

### 6. Evaluación del modelo

In [ ]:
import torch
import random

In [ ]:
model_path = "./drive/MyDrive/training/models/sft_lora_multiples_of_three"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
# Función para generar texto
def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.01
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
numeros_a_palabras = {
    1: "uno", 2: "dos", 3: "tres", 4: "cuatro", 5: "cinco",
    6: "seis", 7: "siete", 8: "ocho", 9: "nueve"
}

texto_a_numero = {v: k for k, v in numeros_a_palabras.items()}

def generar_serie_objetivo(objetivo):
    serie = []
    suma_actual = 0
    while suma_actual < objetivo:
        numero = random.randint(1, min(9, objetivo - suma_actual))
        serie.append(numero)
        suma_actual += numero
    return serie


def comprobar_suma(cadena):
    numero_inicial = int(cadena.split()[0])

    palabras = cadena.split()[3:]  # Ignorar "es igual a"
    suma_numeros = sum(texto_a_numero.get(palabra, 0) for palabra in palabras)
    return abs(numero_inicial - suma_numeros)

In [ ]:
prompt = "<s>[INST] 13 es igual a [/INST]"
generated_texts = generate_text(prompt, max_length=45, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")
    cadena = f"{text}"

Generated Text 1:
[INST] 13 es igual a [/INST] cinco más ocho 



## Paso 2: Fine tuning de un modelo con dataset tipo 'instruct' para números impares

### 1. Generación de nuevos números de entrenamiento que no sean múltiplos de 3

In [ ]:
# Paso 1: Generar un número aleatorio entre 10 y 50 que no sea múltiplo de 3
def generar_numero_aleatorio_no_multiplo_de_tres():
    numero = random.randint(10, 50)
    while numero % 3 == 0:
        numero = random.randint(10, 50)
    return numero

# Diccionario para convertir números a palabras
numeros_a_palabras = {
    1: "uno", 2: "dos", 3: "tres", 4: "cuatro", 5: "cinco",
    6: "seis", 7: "siete", 8: "ocho", 9: "nueve"
}

# Paso 2 y 3: Generar una serie de números cuya suma sea el número aleatorio
def generar_serie_objetivo(objetivo):
    serie = []
    suma_actual = 0

    while suma_actual < objetivo:
        numero = random.randint(1, min(9, objetivo - suma_actual))
        serie.append(numero)
        suma_actual += numero

    return serie

# Paso 4 y 5: Generar múltiples cadenas con números no múltiplos de 3
def generate_lots_of_strings(n=10):
    list_of_strings = []
    for _ in range(n):
        # Generar un número aleatorio entre 10 y 50 que no sea múltiplo de 3
        numero_aleatorio = generar_numero_aleatorio_no_multiplo_de_tres()
        serie_aleatoria = generar_serie_objetivo(numero_aleatorio)
        serie_en_palabras = " más ".join(numeros_a_palabras[numero] for numero in serie_aleatoria)
        s = '{"text": "<s>[INST] %d es igual a [/INST] %s </s>"}' % (numero_aleatorio, serie_en_palabras)
        list_of_strings.append(s)

    # Guardar la lista de cadenas en un archivo
    with open("./drive/MyDrive/training/data/list_of_strings_no_multiples_of_3.jsonl", "w") as file:
        file.write("\n".join(list_of_strings))

# Generar las cadenas
generate_lots_of_strings(150)


### 2. Cargar el modelo

In [ ]:
model_path = "./drive/MyDrive/training/models/sft_lora_multiples_of_three/"
model_name = "sft_lora_multiples_of_three"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


### 3. Cargar los nuevos datos de entrenamiento

In [ ]:
train_dataset = load_dataset('json', data_files='./drive/MyDrive/training/data/list_of_strings_no_multiples_of_3.jsonl', split="train")


def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=64)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])


wandb.init(
    project='LLM_training',
    name=model_name + '_autoregressive_instruct_fine_tuning_non_multiples_of_three'
)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find nb01-instruct.ipynb.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: susanasrez (data2023). Use `wandb login --relogin` to force relogin


### 4. Configuración del entrenamiento mediante LoRA y SFT

In [ ]:
# Configuración de LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # rango de LoRA
    lora_alpha=16,  # hiperparámetro de LoRA
    lora_dropout=0.1,  # dropout de LoRA
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']  # módulos objetivo para aplicar LoRA
)

model = get_peft_model(model, lora_config)

# Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=1e-4,
    fp16=True,
    evaluation_strategy="no",
    eval_steps=10_000,
    report_to="wandb"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in t

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
100,1.632200
200,0.632800
300,0.597700
400,0.545400
500,0.484600
600,0.396400
700,0.268800


TrainOutput(global_step=750, training_loss=0.6236700134277344, metrics={'train_runtime': 779.7464, 'train_samples_per_second': 1.924, 'train_steps_per_second': 0.962, 'total_flos': 4107819810816000.0, 'train_loss': 0.6236700134277344, 'epoch': 10.0})

### 5. Guardar el modelo

In [ ]:
wandb.finish()

trainer.save_model("./drive/MyDrive/training/models/sft_lora_NON_multiples_of_three")
tokenizer.save_pretrained("./drive/MyDrive/training/models/sft_lora_NON_multiples_of_three")

train/epoch,▁▂▃▄▅▆▇█
train/global_step,▁▂▃▄▅▆▇█
train/grad_norm,▄▄▇▆▅█▁
train/learning_rate,█▇▆▄▃▂▁
train/loss,█▃▃▂▂▂▁
total_flos,4107819810816000.0
train/epoch,10
train/global_step,750
train/grad_norm,1.58736
train/learning_rate,1e-05
train/loss,0.2688


('./drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/tokenizer_config.json',
 './drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/special_tokens_map.json',
 './drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/tokenizer.model',
 './drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/added_tokens.json',
 './drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/tokenizer.json')

### 6. Evaluación del modelo

In [ ]:
import torch

In [ ]:
model_path = "./drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
prompt = "<s>[INST] 13 es igual a [/INST]"
generated_texts = generate_text(prompt, max_length=45, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")
    cadena = f"{text}"

Generated Text 1:
[INST] 13 es igual a [/INST] cinco más cuatro más tres más uno 



In [ ]:
prompt = "<s>[INST] 17 es igual a [/INST]"
generated_texts = generate_text(prompt, max_length=45, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")
    cadena = f"{text}"

Generated Text 1:
[INST] 17 es igual a [/INST] nueve más ocho 



## Paso 3: Evaluación con nuevos datos

### 1. Generar los datos de evaluación

In [34]:
import json

In [35]:
def extract_numbers(file):
    numbers = set()
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            text = data["text"]
            num = int(text.split(" es igual a ")[0].split("[INST]")[-1].strip())
            numbers.add(num)
    return numbers

In [36]:
file_jsonl_multiples = "/content/drive/MyDrive/training/data/list_of_strings.jsonl"
file_jsonl_no_multiples = "/content/drive/MyDrive/training/data/list_of_strings_no_multiples_of_3.jsonl"

numbers_multiples = extract_numbers(file_jsonl_multiples)
numbers_no_multiples = extract_numbers(file_jsonl_no_multiples)

training_numbers = numbers_multiples | numbers_no_multiples

In [37]:
min_val = 1
max_val = 100
num_to_generate = 100
new_numbers = set(range(min_val, max_val + 1)) - training_numbers

output_file = "/content/drive/MyDrive/training/data/new_numbers_for_evaluation.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for num in sorted(new_numbers):
        prompt = f"<s>[INST] {num} es igual a [/INST]"
        json.dump({"text": prompt}, f, ensure_ascii=False)
        f.write("\n")

print(f"Se generaron {len(new_numbers)} números nuevos y se guardaron en {output_file}.")


Se generaron 59 números nuevos y se guardaron en /content/drive/MyDrive/training/data/new_numbers_for_evaluation.jsonl.


### 2. Cargar el modelo

In [20]:
model_path = "./drive/MyDrive/training/models/sft_lora_NON_multiples_of_three/"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

### 3. Evaluar el modelo

Para evaluar el modelo, debe ser que la suma de los números que de como salida, sea la suma del número real. Por tanto, para cada número se sumará 1 si corresponde a la suma o se guardaran las diferencias y los números en los que el modelo no ha predicho con exactitud.

In [38]:
import torch

In [39]:
def check_sum(prompt):
    text_to_number = {
        "uno": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9
    }

    initial_number = int(prompt.split("[INST]")[1].split("es igual a")[0].strip())
    words = prompt.split("[/INST]")[1].strip().split(" más ")
    number_sum = sum(text_to_number[word] for word in words if word in text_to_number)
    difference = initial_number - number_sum

    return abs(difference)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
evaluation_file = "/content/drive/MyDrive/training/data/new_numbers_for_evaluation.jsonl"
results = {
    "correct": 0,
    "incorrect": 0,
    "details": []
}

model.eval()
with open(evaluation_file, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        prompt = data["text"]

        generated_text = generate_text(prompt)[0]
        print(generated_text)

        difference = check_sum(generated_text)
        if difference == 0:
            results["correct"] += 1
        else:
            results["incorrect"] += 1
            results["details"].append({"prompt": prompt, "generated_text": generated_text, "difference": difference})


[INST] 1 es igual a [/INST] uno más uno más tres más uno más dos más tres más uno más cuatro más uno 
[INST] 2 es igual a [/INST] tres más uno más uno más uno más uno más uno más uno más uno más uno 
[INST] 3 es igual a [/INST] seis más uno más uno más cuatro más uno más uno 
[INST] 4 es igual a [/INST] uno más dos más uno más uno 
[INST] 5 es igual a [/INST] uno más dos más uno más uno 
[INST] 6 es igual a [/INST] uno más cuatro más uno 
[INST] 7 es igual a [/INST] tres más cuatro 
[INST] 8 es igual a [/INST] tres más cuatro más uno más uno 
[INST] 9 es igual a [/INST] uno más cuatro más tres más uno 
[INST] 51 es igual a [/INST] uno más nueve más nueve más dos más cinco más ocho más cinco más cuatro más tres más uno más tres 
[INST] 52 es igual a [/INST] uno más nueve más tres más cuatro más dos más tres más uno más ocho más dos más cuatro más dos 
[INST] 53 es igual a [/INST] dos más dos más cinco más tres más dos más siete más cinco más siete más dos más tres más dos más dos 
[INST

In [42]:
accuracy = results["correct"] / (results["correct"] + results["incorrect"]) * 100
print(f"Precisión: {accuracy:.2f}%")
print(f"Número correcto: {results['correct']}")
print(f"Número incorrecto: {results['incorrect']}")

results_file = "/content/drive/MyDrive/training/data/evaluation_results.json"
with open(results_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Resultados guardados en {results_file}")

Precisión: 6.78%
Número correcto: 4
Número incorrecto: 55
Resultados guardados en /content/drive/MyDrive/training/data/evaluation_results.json
